# BYO container with SageMaker

In [ ]:
import os
import numpy as np
import sagemaker
import boto3
import time
import math
import json
import re
import pandas as pd
from sagemaker.session import s3_input
from sagemaker.estimator import Estimator

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

client = boto3.client('sagemaker')
region = boto3.Session().region_name
account = boto3.client('sts').get_caller_identity().get('Account')

In [ ]:
bucket_name = 'reinvent-409'
repo_name = 'reinvent-409'
image_tag = 'pytorch'

In [ ]:
output_path = 's3://{}/'.format(bucket_name)
train_path = 's3://{}/cifar10-dataset/cifar-10-batches-py'.format(bucket_name)
eval_path = 's3://{}/cifar10-dataset/cifar-10-batches-py'.format(bucket_name)
image = '{}.dkr.ecr.{}.amazonaws.com/{}:{}'.format(account, region, repo_name, image_tag)

In [ ]:
hyperparams={
    'max_learning_rate':0.4,
    'data_aug_cutout_size':5,
    'batch_size':128,
    'momentum':0.9,
    'batch_norm':"true",
    'epochs':10
}

In [ ]:
estimator = Estimator(
    image_name=image,
    role=role,
    train_instance_count=1,
    train_instance_type='local_gpu',
    output_path=output_path,
    base_job_name='reinvent',
    hyperparameters=hyperparams,
    metric_definitions=[{'Name': 'test_acc', 'Regex': 'test_acc:([0-9\\.]+)'}])

estimator.fit({'training': train_path, 'eval': eval_path})